In [3]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')
from natsort import natsorted
import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A
import time
today = time.strftime("%Y_%m_%d")
from enrich_dimensions.rounds import rounds, rounding_inch_feet,rounding_lbs,rounding_w,rounding_oz, rounding_lb,rounding_gal, re_extract, curate, round_string_float, clean_data,reg_ex,rounding_period_after_unit 
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

# parameters
customer_id = '66'
formatted_attribute = 'Weight Capacity'
customer_name='%bedbathandbeyond%'
buckets = """Office Chairs"""

attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

import json
dfs = query_from_file(file_name='./query/custom_fields_no_att.sql', params=params)
custom_field_df=pd.json_normalize(dfs['custom_fields'])
fields = ['LONG_DESCRIPTION']
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df], axis = 1)
import time
today = time.strftime("%Y_%m_%d")
print(today)
print(len(df))
# reg_ex(df)

del df['url']
del df['image_url']
del df['value']
del df['product_id']
del df['bucket_id']
df.drop_duplicates(subset='external_id', inplace=True)
# removeF
import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])

2023_01_22
9
[]


In [2]:
# Use KewWordProcessor to find a list of values for a single bucket 
df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
list_values=df_strategy['value'].to_list()

kwp=KeywordProcessor()
## JUNK
# print(kwp.add_keywords_from_list(list_values))
# # Extract Key Words
# df['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #

# df['matches'] = df['long_desc'].apply(lambda x: kwp.extract_keywords(x))
# df['matches'].explode().value_counts() 
list_values 

['100 - 149 lbs',
 '1 - 49 lbs',
 '150 - 199 lbs',
 '200 lbs and Greater',
 '50 - 99 lbs']

In [3]:
import re

metric='lbs'
range_string = """
1 - 49 lbs
50 - 99 lbs
100 - 149 lbs
150 - 199 lbs
200 lbs and Greater
 """

range_params = {}
for range_entry in range_string.split('\n'):
    range_nums = re.findall('\d+', range_entry)
    if len(range_nums) > 0: 
        range_params[tuple(map(int, range_nums))] = range_entry.strip()


def  range_app(num_lst):
    updated_labels = []
    for num in num_lst:
        num = float(num)
        for range_param, range_label in range_params.items():
            if len(range_param) == 1:
                if num >= range_param[0]:
                    updated_labels.append(range_label)
            else:
                if num >= range_param[0] and num <= range_param[1]:
                    updated_labels.append(range_label)
    return updated_labels

In [4]:
# Drop duplicate external_ids
import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])
# df=df[(df['external_id'].astype(str)!='200045713')&(df['external_id'].astype(str)!='200045715')&(df['external_id'].astype(str)!='200045714')]

[]


In [5]:
# regex pattern for Weight Capacity
regex_pattern =r"(?i)(\d+.?lb)|()"
df['lb'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
df_lb=df[df['lb'].astype(str)!='[]']

regex_pattern =r"(?i)(suitable\s?.{0,25}\d+\s?lb)|(support.{0,15}w?e?i?g?h?i?n?g?\s?\d+\s?lb)|((?<!weighs )(?<!\))(?<!\.)\d+(?:\.\d+)?(?:\,\d+)?\s?lb\.?\W{0,2}\s?p?o?u?n?d?\s?m?a?x?i?m?u?m?\s?s?t?a?t?i?c?\s?l?o?a?d?\s?d?i?s?t?r?i?b?u?t?e?d?\s?w?e?i?g?h?t?\s?s?e?a?t?i?n?g?\s?capacity(?! \d+ lb))|(\s\d+\s?.?\s?m?a?x?i?m?u?m?\s?weight capacity)|(supports.{0,30}\s?up\s?to\s?\d+\s?lb)|(seats.{0,15}\d+\s?lb(?!\s?and.?up))|(maxi?m?u?m?\s?.{0,15}w?e?i?g?h?t?.?\s?\d+\s?lb)|(weight.?capacity.?o?f?.?.?\s?\d+\s?lb)|((?<!weighs.)\d+.?\d*.?l?b?\s?to\s?\d+.?\d*\s?lb)|((?<!weighs.)\d+.?\d*.?l?b?\s?-\s?\d+.?\d*\s?lb)|(up.?to.?\d+\.?\d*.?lb)|(supports.{0,30}\s?up\s?to\s?\d+\s?lb)|(p?o?u?n?d?\s?weight capacity\s?of\s?.?\s?\d+.?\d*\s?l?b?)|(store.?\s?up\s?to\s?\d+\s?lb)|(users\s?up\s?to\s\d+\s?lb)|(hold.?\s?up\s?to\s?\d+\s?lb)|(Rated\s?up\s?to\s?\d+\s?lb)|(\d+\s?lb.?.?\s?break\s?strength)|()"                  
# regex_pattern=r"(?i)(suitable\s?.{0,25}\d+\s?lb)|(support.{0,15}w?e?i?g?h?i?n?g?\s?\d+\s?lb)|((?<!weighs )(?<!\))(?<!\.)\d+(?:\.\d+)?(?:\,\d+)?\s?lb\.?\W{0,2}\s?p?o?u?n?d?\s?m?a?x?i?m?u?m?\s?s?t?a?t?i?c?\s?l?o?a?d?\s?d?i?s?t?r?i?b?u?t?e?d?\s?w?e?i?g?h?t?\s?s?e?a?t?i?n?g?\s?capacity(?! \d+ lb))|(\s\d+\s?.?\s?m?a?x?i?m?u?m?\s?weight capacity)|(supports.{0,30}\s?up\s?to\s?\d+\s?lb)|(seats.{0,15}\d+\s?lb(?!\s?and.?up))|(weight.?capacity.?o?f?.?.?\s?\d+\s?lb)|((?<!weighs.)\d+.?\d*.?l?b?\s?to\s?\d+.?\d*\s?lb)|((?<!weighs.)\d+.?\d*.?l?b?\s?-\s?\d+.?\d*\s?lb)|(up.?to.?\d+\.?\d*.?lb)|(supports.{0,30}\s?up\s?to\s?\d+\s?lb)|(p?o?u?n?d?\s?weight capacity\s?of\s?.?\s?\d+.?\d*\s?l?b?)|(store.?\s?up\s?to\s?\d+\s?lb)|(users\s?up\s?to\s\d+\s?lb)|(hold.?\s?up\s?to\s?\d+\s?lb)|(Rated\s?up\s?to\s?\d+\s?lb)|(\d+\s?lb.?.?\s?break\s?strength)|(max(?:imum)?\s?\w{0,15}\s?(?:weight)?\W?\s?(?<!of )\d+\s?lb)|()"
df_lb['matches'] = df_lb['long_desc'].apply(lambda x: re_extract(regex_pattern, x))

In [6]:
# Cleaning Data 
regex_lbs=r"(?i)(\d+(?:\.\d+)?\s?lb)|()"
df_lb['matchezs']=df_lb['matches'].apply(lambda x: re.sub(r'(?i)(\\{0,3}xa0)','',str(x)))
df_lb['matchezs']=df_lb['matchezs'].apply(lambda x: re.sub(r'(?<=)\,(?=\d)','',str(x)))
df_lb['a']=df_lb['matchezs'].apply(lambda x: re_extract(regex_lbs, str(x)))

df_newest=df_lb[(df_lb['a'].astype(str)!='[]')]#&(df_lb['a'].astype(str)!='[0lb]')]
df_newest=df_newest[df_newest['a'].astype(str)!='[]']
print(len(df_newest))
# df_newest['aa_two']=df_newest['aa_one'].apply(lambda x: natsorted(x))
df_newest['matchez']=df_newest['a'].apply(lambda x: re.sub(r'(?i)(Maximum(?:\s|\\{0,3}xa0)seating(?:\s|\\{0,3}xa0)capacity)|(\\{0,3}xa0)|(Maximum)|(weight)|(capacity)|(up to)|(suitable)|(for ages)|(\d years and up)|()','',str(x))).apply(lambda x: re.sub(r'lb\.','lb',str(x))).apply(lambda x: re.sub(r'(?i)\s?\.\s?lb|(?<!l)b','lb',str(x))).apply(lambda x: re.sub(r'(?i)\s?\.\s?lb|(?<!l)b','lb',str(x))).apply(lambda x: re.sub(r'(?<=\d)lb',' lb',str(x)))
df_newest['matchez']=df_newest['matchez'].apply(lambda x: re.sub(r'  ',' ',str(x))).apply(lambda x: re.sub(r'(?i)\s?lb\s?',' lb',str(x))).apply(lambda x: re.sub(r'((?<!\d)0\.\d+)','1',str(x)))
df_newest['matchez']=df_newest['matchez'].apply(lambda x: re.sub(r'\[','["',str(x))).apply(lambda x: re.sub(r'\]','"]',str(x))).apply(lambda x: re.sub(r',','","',str(x))).apply(lambda x: re.sub(r'""','"',str(x))).apply(lambda x: re.sub(r"'",'',str(x)))#.apply(lambda x: re.sub(r'(?i)\s?\.\s?lb|(?<!l)b','lb',str(x)))
curate(df_newest, 'matchez', formatted_attribute)
df_newest['matchez']=df_newest['matchez'].apply(lambda x: re.sub(r'(?i)\s?l?b\s?|i|n|b','',str(x)))
df_newest

df_newest['matchezzz']=df_newest['matchez'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)])).apply(range_app).apply(lambda x: re.sub(r"\['",'["',str(x))).apply(lambda x: re.sub(r"'\]",'"]',str(x))).apply(lambda x: re.sub(r"'\s?,\s?'",'","',str(x))).apply(lambda x: re.sub(r"'\[",'[',str(x))).apply(lambda x: re.sub(r"\]'",']',str(x))).apply(lambda x: re.sub(r'\,\s',',',str(x))).apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)])) .apply(lambda x: re.sub(r"\['",'["',str(x))).apply(lambda x: re.sub(r"'\]",'"]',str(x))).apply(lambda x: re.sub(r"'\s?,\s?'",'","',str(x))).apply(lambda x: re.sub(r"'\[",'[',str(x))).apply(lambda x: re.sub(r"\]'",']',str(x)))

df_newest[curation_col]=df_newest['matchezzz']
df_newest[curation_col]=df_newest[curation_col].apply(lambda x: re.sub(r'(?i)((?<!l)b)|(lb)','lb',str(x)))

rounds(df_newest,curation_col)
rounding_period_after_unit(df_newest,curation_col,'^\[\]lb\s\dandgreaterG"s\-\,,\.\]')


# print(len(df_newest))
# rounding_lbs(df_newest,curation_col,'ac-km-z') 
# df_newest=df_newest[df_newest['rounding'].astype(str)!='[]']
# print(len(df_newest))


76
Repeated External IDs: []
Number of SKUs that need Rounding: 0


Empty DataFrame
Columns: [buckets, external_id, product_name, long_desc, LONG_DESCRIPTION, lb, matches, matchezs, a, matchez, ones, twos, threes, fours, fives, Q:weight_capacity, matchezzz, rounding]
Index: []

In [7]:
# print(len(df_newest[df_newest['newest_matches'].astype(str)!='[]']))
# df_newest=df_newest[df_newest['newest_matches'].astype(str)!='[]']

In [8]:
matchesnewest=df_newest[['external_id',curation_col]]
print(len(df_newest))

76


In [9]:
# df_newest[df_newest['rounding'].astype(str)!='[]']

# Na

In [10]:
regex_pattern_na=r'(?i)(?i)(suitable\s?.{0,25}\d+\s?lb)|(support.{0,15}w?e?i?g?h?i?n?g?\s?\d+\s?lb)|((?<!weighs )(?<!\))(?<!\.)\d+(?:\.\d+)?(?:\,\d+)?\s?lb\.?\W{0,2}\s?p?o?u?n?d?\s?m?a?x?i?m?u?m?\s?s?t?a?t?i?c?\s?l?o?a?d?\s?d?i?s?t?r?i?b?u?t?e?d?\s?w?e?i?g?h?t?\s?s?e?a?t?i?n?g?\s?capacity(?! \d+ lb))|(\s\d+\s?.?\s?m?a?x?i?m?u?m?\s?weight capacity)|(supports.{0,30}\s?up\s?to\s?\d+\s?lb)|(seats.{0,15}\d+\s?lb(?!\s?and.?up))|(maxi?m?u?m?\s?.{0,15}w?e?i?g?h?t?.?\s?\d+\s?lb)|(weight.?capacity.?o?f?.?.?\s?\d+\s?lb)|((?<!weighs.)\d+.?\d*.?l?b?\s?to\s?\d+.?\d*\s?lb)|((?<!weighs.)\d+.?\d*.?l?b?\s?-\s?\d+.?\d*\s?lb)|(up.?to.?\d+\.?\d*.?lb)|(supports.{0,30}\s?up\s?to\s?\d+\s?lb)|(p?o?u?n?d?\s?weight capacity\s?of\s?.?\s?\d+.?\d*\s?l?b?)|(store.?\s?up\s?to\s?\d+\s?lb)|(users\s?up\s?to\s\d+\s?lb)|(hold.?\s?up\s?to\s?\d+\s?lb)|(Rated\s?up\s?to\s?\d+\s?lb)|(\d+\s?lb.?.?\s?break\s?strength)'   
df['matches_na'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_na, x))

df_na=df[(df['matches_na'].astype(str)=='[]')]
df_na[curation_col]='n/a'
print(len(df_na))
match_na=df_na[['external_id',curation_col]]

86


In [11]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchesnewest))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(match_na))

matchesnewest
match_na


In [12]:
print(len(df))
print(len(matchesnewest)+len(match_na))

195
162


In [13]:
# error

# send to the folder for upload

In [14]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-{today}-matches.csv',index=False) 

looks_good('Bed Bath & Beyond', attribute, df, matchesnewest,today)

In [15]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches,today): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-{today}-matches.csv',index=False) 

looks_good('Bed Bath & Beyond', attribute, df, match_na,today)